In [1]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), os.pardir))

In [2]:
%autoreload 2
%config InlineBackend.figure_format = 'svg'
from pymcts.uct import UCTNode
from pymcts.game.trivial import TrivialState
from pymcts.game.tic_tac_toe import TicTacToeState

In [3]:
win_state = TrivialState()
loss_state = TrivialState(result={1: 0})
root_state = TrivialState(result=None, moves={1: win_state, 2: loss_state})

trivial_root = UCTNode(root_state)
for _ in range(100):
    trivial_root.mc_round()

trivial_root

UCTNode(M: None, P1, Wins/Visits: 94.0/100, |U|: 0, S: [1, 2], [
    UCTNode(M: 2, P1, Wins/Visits: 0.0/6, |U|: 0, S: {1: 0}),
    UCTNode(M: 1, P1, Wins/Visits: 94.0/94, |U|: 0, S: {1: 1.0})
])

Above we see a trivial two-stage game where Player 1 always 'wins' following taking move 1, and always loses after taking move 2.

We then set up a UCT tree and run some iterations. One consequence of the UCT algorithm here is that an always-losing node will get revisited with exponentially declining frequency as the number of iterations increase. This shows that UCT focuses its visits (during the 'selection' part of MCTS) on subtrees that perform well, and hence searches deeper into parts of the game tree that see stronger play.

In [4]:
s = TicTacToeState()

root = UCTNode(s)
for _ in range(int(3E5)):
    root.mc_round()
root.wins / root.visits

0.46468

Here we repeat the exercise with Tic Tac Toe. We know that perfect play always results in a tie (score = 0.5); in general, UCB scores should underestimate the true value of a node and converge from below as the number of iterations rises.

In [5]:
root.best_move()

(1, 1)

As expected, this tree identifies that the best move for the first player is to go in the center.

Now, let's try to show what the game tree looks like:

In [105]:
import pandas as pd
import graphistry
import igraph
graphistry.register(key=open('../../../.graphistry_api_key', 'r').read())

ig = to_igraph(root, max_depth=5)

layout = ig.layout_reingold_tilford_circular(mode='out', root=[0])
layout_df = pd.DataFrame(layout.coords)
layout_df.columns = ['x', 'y']

binder = graphistry.bind(source='src',
                destination='dst',
                node='nodeid',
                point_title='state',
                point_size='path_prob',
                edge_weight='path_prob',
                edge_title='move')
e, v = binder.igraph2pandas(ig)

# Make multi-line comments render properly in Graphistry
v['state'] = v['state'].apply(lambda s: html.escape(s).replace('\n', '<br />'))

v_pos = pd.concat([v, layout_df], axis=1)
binder.nodes(v_pos).edges(e).settings(url_params={'play':10000}).plot()

Above, game tree nodes are sized by the marginal rate at which that they will be sampled in the immediate future; stronger nodes/subtrees will appear larger. As expected, the line of play where X opens in center and O then plays for a draw in a corner dominates our current search.

We can also use ```igraph``` directly to visualize the tree:

In [115]:
import igraph as ig
from pymcts.drawing.mct_graph import to_igraph

#layout='kk'
layout=g.layout_reingold_tilford(mode="out", root=[0])
g = to_igraph(root, max_depth=3)
ig.plot(g, layout=layout)